In [32]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [35]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())
sc = spark.sparkContext

In [45]:
# тащим файл data в оперативку в формате rdd
rdd_u=sc.textFile('/labs/laba01/ml-100k/u.data')
rdd_u.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [47]:
rdd_u_split=rdd_u.map(lambda x: x.split('\t'))
rdd_u_split.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [48]:
rdd_u_split_cut=rdd_u_split.map(lambda z: z[1:3])
rdd_u_split_cut.take(10)

[['242', '3'],
 ['302', '3'],
 ['377', '1'],
 ['51', '2'],
 ['346', '1'],
 ['474', '4'],
 ['265', '2'],
 ['465', '5'],
 ['451', '3'],
 ['86', '3']]

In [50]:
rdd_u_split_cut_filtered = rdd_u_split_cut.filter(lambda y: y[0] == my_film_id)
rdd_u_split_cut_filtered.take(10)

[['302', '3'],
 ['302', '4'],
 ['302', '4'],
 ['302', '4'],
 ['302', '3'],
 ['302', '5'],
 ['302', '3'],
 ['302', '4'],
 ['302', '5'],
 ['302', '4']]

In [52]:
rdd_u_split_cut_filtered_formated = rdd_u_split_cut_filtered.map(lambda n: [int(n[1]),n[0]])
rdd_u_split_cut_filtered_formated.take(10)

[[3, '302'],
 [4, '302'],
 [4, '302'],
 [4, '302'],
 [3, '302'],
 [5, '302'],
 [3, '302'],
 [4, '302'],
 [5, '302'],
 [4, '302']]

In [53]:
rdd_u_split_cut_filtered_formated.countByKey().items()

dict_items([(3, 46), (4, 119), (5, 120), (2, 10), (1, 2)])

In [58]:
hist_film=list(dict(sorted(rdd_u_split_cut_filtered_formated.countByKey().items())).values())

AttributeError: 'list' object has no attribute 'show'

In [56]:
rdd_u_split_cut_formated_all = rdd_u_split_cut.map(lambda n: [int(n[1]),n[0]])
rdd_u_split_cut_formated_all.take(10)

[[3, '242'],
 [3, '302'],
 [1, '377'],
 [2, '51'],
 [1, '346'],
 [4, '474'],
 [2, '265'],
 [5, '465'],
 [3, '451'],
 [3, '86']]

In [64]:
hist_all=list(dict(sorted(rdd_u_split_cut_formated_all.countByKey().items())).values())

In [62]:
hist_film

[2, 10, 46, 119, 120]

In [65]:
hist_all

[6110, 11370, 27145, 34174, 21201]

In [66]:
result={'hist_film':hist_film,'hist_all':hist_all}
result

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [68]:
# одним шагом
RDD=sorted(sc.textFile('/labs/laba01/ml-100k/u.data')\
    .map(lambda x: x.split('\t'))\
    .map(lambda z: z[1:3][::-1])\
    .map(lambda y: [y[0],[1,1 if y[1] == my_film_id else 0]])\
    .reduceByKey(lambda x,y: [x[0]+y[0],x[1]+y[1]]).collect())

hist_film3 =[ i[1][1] for i in RDD]
hist_all3 = [ i[1][0] for i in RDD]
result3={'hist_film':hist_film3,'hist_all':hist_all3}
result3

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [70]:
# самый понятный вариант
hist_film2=list(dict(sorted(sc.textFile('/labs/laba01/ml-100k/u.data')\
    .map(lambda x: x.split('\t'))\
    .map(lambda z: z[1:3])\
    .map(lambda n: [n[1],n[0]])\
    .filter(lambda y: y[1] == my_film_id).countByKey().items())).values())

hist_all2=list(dict(sorted(sc.textFile('/labs/laba01/ml-100k/u.data')\
    .map(lambda x: x.split('\t'))\
    .map(lambda z: z[1:3])\
    .map(lambda n: [n[1],n[0]])\
    .countByKey().items())).values())

result2={'hist_film':hist_film2,'hist_all':hist_all2}
result2

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [30]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [31]:
!hdfs dfs -ls /labs/laba01/ml-100k/README

-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README


In [29]:
!hdfs dfs -ls /materials

ls: `/materials': No such file or directory


In [23]:
my_film_id = '302'

In [26]:
sc.stop()

In [27]:
rdd_u=sc.textFile('/labs/laba01/ml-100k/u.data')

AttributeError: 'NoneType' object has no attribute 'sc'

In [ ]:
spark.stop()

In [73]:
with open('lab01.json', 'w') as fp:
    json.dump(result3, fp)